local senitivity hash 방법을 이용하여 ID를 기준으로 유사한 데이터(ID, PW쌍)를 모은뒤

패스워드들간 유사도를 JARO Winkler 알고리즘을 통해 분류하였고 threshold 0.8을 넘는 데이터만을

dataframe으로 정리하였다. 이 데이터 프래임은 ./jdh/korean/data1~4.csv로 각각저장하였고 이를 concat한 데이터 

./jdh/korean/dataset.csv의 크기는 약110만 이었다

In [25]:
import re
import time
import string
from datasketch import MinHash, MinHashLSHForest

import os
import pickle
import random
import pandas as pd
from pyjarowinkler import distance

In [91]:
def preprocess(text):
    num_gram = 4
    tokens = [text[i:i+num_gram] for i in range(len(text)-num_gram+1)]

    return tokens

def get_forest(data, perms):
    start_time = time.time()
    
    minhash = []
    
    for text in data['ID']:
        tokens = preprocess(text)
        m = MinHash(num_perm=perms)
        for s in tokens:
            m.update(s.encode('utf8'))
        minhash.append(m)
        
    forest = MinHashLSHForest(num_perm=perms)
    
    for i,m in enumerate(minhash):
        forest.add(i,m)
        
    forest.index()
    
    print('It took %s seconds to build forest.' %(time.time()-start_time))
    
    return forest

#def predict(text, database, perms, num_results, forest):
#def predict(text, database, perms, num_results, forest):
    start_time = time.time()
    
    tokens = preprocess(text)
    m = MinHash(num_perm=perms)
    for s in tokens:
        m.update(s.encode('utf8'))
        
    idx_array = np.array(forest.query(m, num_results))
    if len(idx_array) == 0:
        return None # if your query is empty, return none
    
    #result = database.iloc[idx_array]['id']
    #result_pw = database.iloc[idx_array]['password']
    result = idx_array
    
    #print('It took %s seconds to query forest.' %(time.time()-start_time))
    
    return result

def predict(text, perms, num_results, forest):

    start_time = time.time()
    
    tokens = preprocess(text)
    m = MinHash(num_perm=perms)
    for s in tokens:
        m.update(s.encode('utf8'))
        
    idx_array = np.array(forest.query(m, num_results))
    if len(idx_array) == 0:
        return None # if your query is empty, return none
    
    result = idx_array
    
    return result

def hash_list(values):
    total = '0'
    for i in range(len(values)):
        total += values[i]    
    result = hash(total)
    #print(total)
    
    return(result)

def lsh(num):
    
    a= data['ID'][num]
    b = data['PW'][num]
    refer = [a,b]
    
    idx = num
    compare_id = refer[0]
    
    predict_result = predict(compare_id, permutations, num_recommendations, forest)
    
    #lsh_data = data.iloc[sorted(predict_result)]    
    return(predict_result)

def Hashing(hash_list):
    total = '0'
    for i in range(len(hash_list)):
        total += hash_list[i]    
    result = hash(total)
    #print(total)
    
    return(result)

def PWset(idx):
    reference = data['PW'][idx]
    
    result  = lsh(idx)
    temp_df = data.loc[sorted(result)]
    
    PW = list(temp_df['PW'].values)
    PWset = list()
    
    for password in PW:
        temp = distance.get_jaro_distance(reference, password)
        #print(reference, password)
        if 1>= temp > threshold:
            PWset.append(password)
    PWset = list(set(PWset))
    
    if len(PWset)>1:
        return PWset

In [5]:
def SaveForest(forest, seq):
    
    save_path = './LSH_Pickle/'+'s/'+'forest_s_'+str(seq) +'.pickle'
    with open (save_path, 'wb') as f:
        pickle.dump(forest,f)
    print(seq, 'forest_saved')
    
def SavePWset(data, seq):
        
    data = data.dropna(how='any')
    data['HashTag'] = data['PWset'].apply(Hashing)
    data = data.drop_duplicates(subset='HashTag', keep ='first')
    
    
    save_path = './LSH_Data/'+'s/'+'s_'+str(seq) +'.pickle'
    data.to_pickle(save_path)
    print(seq,'Data Pickle Saved')
    print('')

In [135]:
#Number of Permutations

threshold = 0.8
permutations = 32
num_recommendations = 30

a(24), b(15), c(15), d(16), e(8), f(7), g(10), h(7), i(6)

j(14), k(13), l(13), m(23), n(9), o(4), p(11), q(1), r(12)

s(25), t(12), u(2), v(7), w(5), x(2), y(3), z(3)

In [ ]:
for i in range(25):
    
    path = './For_LSH_data/s/'+'s_'+str(i)+'.pickle'
    data = pd.read_pickle(path)
    print(data.head())
    
    forest = get_forest(data, permutations)
    
    SaveForest(forest,i)
    
    data = data.assign(index=lambda x: data.index)
    start = time.time()
    data['PWset'] = data['index'].apply(PWset)
    current = time.time() - start
    print('Making PWset consumed time:' ,current)
    
    SavePWset(data, i)
    